In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import migKeyPoint.utils.YAMLtools as yt

In [ ]:
conf = yt.load_configuration('../master_configuration.yaml')['yoloConf']

In [ ]:
conf

### Let's generate our keypoints as we did in part 1. I'll do this more succinctly this time

In [ ]:
class initial_data_processing:
    def __init__(self,datafile):
        self.datafile = datafile #path to simulated ER dataframe
        self.data = self.load_data()
        self.data = self.initial_data_transforms()
        
    def load_data(self):
        df = pd.read_feather(self.datafile)
        return df
    
    def initial_data_transforms(self):
        def find_head(df,i):
            tmp = df.iloc[i]
            indices = pd.Series(tmp['q']).nlargest(3).index.to_numpy()
            return np.median(tmp['x'][indices]),np.median(tmp['y'][indices])
        xs = []
        ys = []
        print('Adding head points to data')
        for i in tqdm(range(0,len(self.data))):
            x,y = find_head(self.data,i)
            xs.append(x)
            ys.append(y)
        self.data['xhead'] = xs
        self.data['yhead'] = ys
        self.data['htlength'] = np.sqrt((self.data['xvtx']-self.data['xhead'])**2+(self.data['yvtx']-self.data['yhead'])**2)
        self.data['xvtx'] = self.data['xvtx']-self.data['x'].apply(lambda x: x.min())+conf['cameraX']//2
        self.data['yvtx'] = self.data['yvtx']-self.data['y'].apply(lambda x: x.min())+conf['cameraY']//2
        self.data['xhead'] = self.data['xhead']-self.data['x'].apply(lambda x: x.min())+conf['cameraX']//2
        self.data['yhead'] = self.data['yhead']-self.data['y'].apply(lambda x: x.min())+conf['cameraY']//2

        self.data['x'] = self.data['x'].apply(lambda x: x-x.min()+conf['cameraX']//2)
        self.data['y'] = self.data['y'].apply(lambda x: x-x.min()+conf['cameraY']//2)
        return self.data

In [ ]:
ERs = initial_data_processing('../simulation/ERs_cont_spectrum_correctE.feather').data

# Generate keypoints like in part 1

In [ ]:
import numpy as np
from scipy.sparse import coo_matrix

class generate_keypoints:
    '''Class takes a sparse image, rotates it so the head and tail are vertically aligned.
    Then it partitions the interval between the head and tail into n_outputs + 2 equally spaced subdivisions
    and records the (x',y') coordinate of either (a) the max intensity [if the mode parameter is set to 'max']
    or (b), the median (x',y') over the 9 most intense pixels in each partition. The code then rotates the set of
    (x',y')s back to the original image orientation, which are our keypoints!'''
    
    def __init__(self,df,i,n_outputs=1,dim=(2048,1152),mode='max'):
        if mode.lower() != 'max' and mode.lower() != 'median':
            raise ValueError("mode must be 'max' or 'median'")
        self.mode = mode.lower()
        evt = df.iloc[i]
        self.n_outputs = n_outputs
        self.center_x = dim[1] // 2
        self.center_y = dim[0] // 2
        self.col = evt['x'].astype('int')
        self.row = evt['y'].astype('int')
        self.data = evt['q']
        self.head = np.array([evt['xhead'],evt['yhead']])
        self.tail = np.array([evt['xvtx'],evt['yvtx']])
        self.rotation_angle = self.get_rotation_angle()
        
        '''Rotation matrices, backward rotation is used to translate track segments back to original'''
        self.forward_rotation = self.generate_rotation_matrix(self.rotation_angle)
        self.reverse_rotation = np.linalg.inv(self.forward_rotation)
        
        '''Rotate head and tail to rotated space'''
        self.rothead = self.rotate_coord(self.head[::-1],self.forward_rotation)
        self.rottail = self.rotate_coord(self.tail[::-1],self.forward_rotation)
        
        #print(self.rotate_coord(self.rothead,self.reverse_rotation))
        '''Generate rotated sparse image'''
        self.rot_im = self.rotate_sparse_image()
        
        '''Get segmented coordinates in rotated space'''
        self.rot_segments = np.array(self.get_segment_coordinates()).T
        self.segments = []
        self.segments.append((evt['xvtx'],evt['yvtx']))
        for coord in self.rot_segments:
            self.segments.append(self.rotate_coord(coord,self.reverse_rotation)[::-1])
        self.segments.append((evt['xhead'],evt['yhead']))
    def get_rotation_angle(self):
        vec = np.array([self.head[0]-self.tail[0],self.head[1]-self.tail[1]])
        theta = np.arctan2(vec[1],vec[0])
        return theta

    def generate_rotation_matrix(self,theta):
        cos_angle = np.cos(theta)
        sin_angle = np.sin(theta)
        
        rotation_matrix = np.array([
            [cos_angle, sin_angle],
            [-sin_angle, cos_angle]
        ])
            
        return rotation_matrix

    def rotate_sparse_image(self):
        sparse_matrix = coo_matrix((self.data, (self.row, self.col)), shape=(1152,2048))
        # Center of the image

        # Translate coordinates to origin
        translated_x = self.col - self.center_x
        translated_y = self.row - self.center_y

        # Apply rotation
        new_coords = np.dot(self.forward_rotation, np.array([translated_x, translated_y]))

        new_x = np.round(new_coords[0] + self.center_x).astype('int')
        new_y = np.round(new_coords[1] + self.center_y).astype('int')

        # Filter out-of-bounds coordinates
        valid_mask = (new_x >= 0) & (new_x < sparse_matrix.shape[1]) & (new_y >= 0) & (new_y < sparse_matrix.shape[0])
        new_x = new_x[valid_mask]
        new_y = new_y[valid_mask]
        new_data = self.data[valid_mask]

        # Create the rotated sparse matrix
        rotated_sparse_matrix = coo_matrix((new_data, (new_y, new_x)), shape=sparse_matrix.shape)
        return rotated_sparse_matrix.toarray().T
    
    def rotate_coord(self,coord,rot):
        original_coordinate = coord

        # Translate coordinate to origin
        translated_x = original_coordinate[1] - self.center_x
        translated_y = original_coordinate[0] - self.center_y

        # Apply rotation
        new_coord = np.dot(rot, np.array([translated_x, translated_y]))

        # Translate back to the original coordinate system
        new_coordinate = (new_coord[1] + self.center_y, new_coord[0] + self.center_x)
        return new_coordinate
    
    def get_segment_coordinates(self):
        n_partitions = self.n_outputs+2
        y_segments = np.linspace(self.rottail[1],self.rothead[1],n_partitions)[1:-1]
        x_segments = []
        for seg in y_segments:
            if self.mode == 'max':
                x_coord = np.median(np.where(self.rot_im[int(np.round(seg)),:] == self.rot_im[int(np.round(seg)),:].max())[0])
            elif self.mode == 'median':
                indices = pd.Series(self.rot_im[int(np.round(seg)),:]).nlargest(9).index.to_numpy()
                x_coord = np.median(indices)
            x_segments.append(x_coord)
        x_segments = np.array(x_segments)
        if np.mean(x_segments) != 575.5 and np.mean(x_segments) != 4:
            return x_segments,y_segments
        else:
            raise ValueError("Bad rotation")

### Let's add keypoints and YOLO information more succinctly than before

In [ ]:
'''There"s an apparent bug in the keypoint generation code where certain kinds of rotations mess things up.
I"ve figured out how to flag these, but haven"t figured out how to fix this. Nevertheless, since this is
meant to be a minimal working example, we won"t sweat this. Because of this bug, there will be a preferred 
direction in the trajectories that do generate. Generally speaking training on a preferred direction will 
bias a machine learning model which we don"t want.'''

ERs = ERs.query('200 > htlength > 50')
ERs.index = [i for i in range(0,len(ERs))]

good_indices = []
coords = {} #dictionary filled with keypoint tuples
for i in range(0,conf['maxNumKeyPoints']):
    coords[i] = []

for i in tqdm(range(0,len(ERs))):
    try:
        a = generate_keypoints(ERs,i,n_outputs=conf['maxNumKeyPoints']-2,mode='median')
        good_indices.append(i)
        for j in range(0,len(a.segments)):
            coords[j].append(a.segments[j])
    except:
        #print("Bad rotation")
        continue

In [ ]:
'''Reduce our dataframe to only include entries where the trajectory generated'''
ERs = ERs.loc[ERs.index.isin(good_indices)] #only keep the events where the loop above didnt fail
ERs.index = [i for i in range(0,len(ERs))]

In [ ]:
'''Make class_index, x, y, width, and height columns'''

ERs['class_index'] = 0

'''Columns "x", "y", and "q" are array-based quantities. We can use pandas"s apply function and
lambda expressions to do array operations rowwise in a succinct way. Don"t worry if this notation
seems cryptic, once you get used to it, it"s very succinct and efficient for array operations
in pandas dataframes'''

ERs['xBB'] = ERs['x'].apply(lambda x: (x.max()+x.min())/2 / conf['cameraX']) #normalized as a fraction of width of image (2048 pixels)
ERs['yBB'] = ERs['y'].apply(lambda x: (x.max()+x.min())/2 / conf['cameraY']) #normalized as a fraction of height of image (1152 pixels)
ERs['width'] = ERs['x'].apply(lambda x: (x.max()-x.min()) / conf['cameraX'])
ERs['height'] = ERs['y'].apply(lambda x: (x.max()-x.min()) / conf['cameraY'])

'''Puts key point tuples into into columns p0 to pN'''
for key in coords.keys():
    ERs['p%s'%(key)] = coords[key]
    
'''Expands the tuples to p0x, p0y, p1x, p1y, etc.'''
# Initialize an empty dictionary to hold the new columns
new_columns = {}

# Iterate over each of the keypoint columns in the DataFrame
for col in ERs.columns[int(-1*conf['maxNumKeyPoints']):]: #apologies that this is
    # Extract x and y components from each column
    ERs[[f'{col}x', f'{col}y']] = pd.DataFrame(ERs[col].tolist(), index=ERs.index)
    # Drop the original column
    ERs.drop(columns=[col], inplace=True)
    
'''Normalize keypoints'''
for i in range(0,conf['maxNumKeyPoints']):
    ERs['p%sx'%(i)] = ERs['p%sx'%(i)]/conf['cameraX']
    ERs['p%sy'%(i)] = ERs['p%sy'%(i)]/conf['cameraY']

# Now onto something new: Making realistic simulation
We can add dark frames to simulation to create noisy images that are more representative of real data. When adding noise, however, there are a couple of effects that need to be simulated:
(1) Gain. We need to scale the light yield of the track relative to noise to match data. Figuring this out is a project in and of itself and something we've already done, so I'll use the scaling I use for Migdal simulation here.
(2) Vignetting: Vignetting is a known effec tin CMOS cameras where the pixel intensity decreases radially outward from the optical center. This is another effect that we've already simulated so I'll include that here as well. Vignetting is a position dependent effect, so we should randomize track locations before applying vignetting.

**Generally speaking, randomizing the locations of tracks along the readout is a good idea, as it makes trained ML models generalize better. If your model is only trained on identifying objects near the center of the readout, it will become *very* good at doing that, and only that. If a model is trained on tracks randomly scattered across the entire readout plane, then it will learn how to look for tracks regardless of position**

In [ ]:
def determine_random_shift(i,border):
    tmp = ERs.iloc[i]
    #Determine track boundaries
    xmin = tmp['x'].min()
    xmax = tmp['x'].max()
    ymin = tmp['y'].min()
    ymax = tmp['y'].max()
    
    #Perform random uniform shifts in x and y
    xshift = np.random.randint(-1*xmin+border,2048-xmax-border)
    yshift = np.random.randint(-1*ymin+border,1152-ymax-border)

    return xshift, yshift

'''Put the shift values in the dataframe so we can then apply them to other columns
in the dataframe, thereby shifting the tracks'''
xshifts = []
yshifts = []
for i in range(0,len(ERs)):
    xshift,yshift = determine_random_shift(i,border = 150)
    xshifts.append(xshift)
    yshifts.append(yshift)
ERs['xshift'], ERs['yshift'] = xshifts, yshifts

In [ ]:
'''Columns that are normalized (width and height stay the same)'''
xcols = ['xBB'] + ['p%sx'%(i) for i in range(0,conf['maxNumKeyPoints'])]
ycols = ['yBB'] + ['p%sy'%(i) for i in range(0,conf['maxNumKeyPoints'])]

'''Unnormalize, shift, and then renormalize'''
for col in xcols:
    ERs[col] = (ERs[col]*conf['cameraX']+ERs['xshift'])/conf['cameraX']

for col in ycols:
    ERs[col] = (ERs[col]*conf['cameraY']+ERs['yshift'])/conf['cameraY']
    
'''Columns that are not normalized'''
xcols = ['xvtx','xhead','x']
ycols = ['yvtx','yhead','y']

for col in xcols:
    ERs[col] = (ERs[col]+ERs['xshift'])

for col in ycols:
    ERs[col] = (ERs[col]+ERs['yshift']) 

In [ ]:
'''Apply intensity scaling to match gains representative of data'''
tqdm.pandas()

'''These gain scaling factors were empirically determined'''
res_fact = 0.115
gf = 5

def calc_light_fraction(dist,QE,f=25,N=0.85,reflect=False):
    L = 0.5*(1 - np.sqrt( 1 - (f/(2*N*dist))*(f/(2*N*dist)) ))
    if reflect:
        L += 0.67*0.67*0.33*L*(dist/(dist + 2 + 2*0.57))**2
    return L*QE*0.34

light_frac = calc_light_fraction(118.7,0.23, reflect = True)

def scale_evt(evt,gain_factor,light_fraction):
    return evt*light_fraction*gain_factor/0.11

def apply_gain_scaling(df,res_fact, gf):
    df['scaled_q'] = df['q'].progress_apply(lambda x: scale_evt(x*np.random.normal(1,res_fact),gain_factor=gf,light_fraction=light_frac))
    df['scaled_q'] = df['scaled_q'].apply(lambda x: np.round(x).astype('int16'))
    df['idx'] = df['scaled_q'].apply(lambda x: np.where(x > 0)[0])
    df['scaled_qsum'] = df['scaled_q'].apply(lambda x: x.sum())

    df['x'] = [df['x'].iloc[i][df['idx'].iloc[i]].astype('int16') for i in tqdm(range(0,len(df)))]
    df['y'] = [df['y'].iloc[i][df['idx'].iloc[i]].astype('int16') for i in tqdm(range(0,len(df)))]
    df['scaled_q'] = [df['scaled_q'].iloc[i][df['idx'].iloc[i]].astype('int16') for i in tqdm(range(0,len(df)))]

    
apply_gain_scaling(ERs,gf=gf,res_fact=res_fact)

In [ ]:
'''Simulate vignetting'''

def apply_vignetting(df,a):
    '''Define centroids. We apply vignetting radially outward'''
    centroidx = 1023
    centroidy = 575
    
    df['x'] = df['x'].apply(lambda x: x.astype('float32'))
    df['y'] = df['y'].apply(lambda x: x.astype('float32'))
    
    '''Compute distance from centroids'''
    df['dist_x'] = df['x'].apply(lambda x: (x-centroidx)*80/2048)
    df['dist_y'] = df['y'].apply(lambda x: (x-centroidy)*80/2048)
    df['dist'] = [np.sqrt(df['dist_x'].iloc[i]**2+df['dist_y'].iloc[i]**2) for i in range(0,len(df))]
    
    '''Apply intensity suppression due to vigentting'''
    df['vignetted_q'] = [df['scaled_q'].iloc[i]/(a**2/(a-df['dist'].iloc[i])**2) for i in range(0,len(df))]
    df['vignetted_q'] = df['vignetted_q'].apply(lambda x: np.round(x).astype('int16'))
    del(df['dist_x'])
    del(df['dist_y'])
    del(df['dist'])
    
    ''' This code removes all charge that's 0 after correcting for vignetting and turning into an integer '''
    df['x'] = df['x'].apply(lambda x: x.astype('int16'))
    df['y'] = df['y'].apply(lambda x: x.astype('int16'))
    
    df['idx'] = df['vignetted_q'].apply(lambda x: np.where(x > 0)[0])
    df['x'] = [df['x'].iloc[i][df['idx'].iloc[i]].astype('int16') for i in tqdm(range(0,len(df)))]
    df['y'] = [df['y'].iloc[i][df['idx'].iloc[i]].astype('int16') for i in tqdm(range(0,len(df)))]
    df['vignetted_q'] = [df['vignetted_q'].iloc[i][df['idx'].iloc[i]].astype('int16') for i in tqdm(range(0,len(df)))]

    
    return df

In [ ]:
ERs = apply_vignetting(ERs,a=95) #a is also empirically determined

### Now we've applied both gain and vignetting scalings to the intensities. The next step is to add noise. Rather than simulating noise, it's best to use dark frames recorded by MIGDAL. Dark frames are those where the camera captures images with the rest of the MIGDAL TPC powered down. The noise distribution varies so we pick dark frames at random from a sample of 200 (we could pick more but loading dense 2048 x 1152 arrays can fill up system memory pretty quickly)

In [ ]:
from skimage import io
dark = io.imread('../darks/MIG_Dark_0V_230803T130339.DARK.0001.MTIFF',plugin='pil')
'''Load masterdark, we will subtract this from the dark frames'''
masterdark = np.load('../darks/master_dark_230803.npz')['arr_0']
dark = dark - masterdark

### Let's downsample our dark frames using 4x4 binning. These aren't sparse arrays, so we have to do this differently than what we've been doing previously. Pytorch's AvgPool function is one way to do this. I prefer using it because it can be computed on a GPU which is much faster than on a CPU

In [ ]:
import torch
import torch.nn as nn
ap = nn.AvgPool2d(kernel_size = (4,4),stride = (4,4),divisor_override = 1)
darkDownSample = ap(torch.tensor(dark)).numpy()

### Remake our training, validation, and test sets with the shifted data. The noise will only be added to images for YOLO to evaluate (we should add it to our dataframe but we don't need to yet)

In [ ]:
dataset_size = len(ERs)
# Shuffle data
ERs = ERs.sample(frac=1,random_state=42) #Random state ensures we get identical shuffles every time for reproducability
ERs.index = [i for i in range(0,len(ERs))] #reset index after shuffling
ERs['index'] = ERs.index
data = {} #dictionary storing train, validation, and test datasets
data['train'] = ERs[:int(dataset_size*0.7)]
data['valid'] = ERs[int(dataset_size*0.7):int(dataset_size*0.9)]
data['test'] =  ERs[int(dataset_size*0.9):]
print('Train set size: %s\nval set size : %s\ntest set size: %s\nsum : %s\ndataset size: %s'%(len(data['train']),len(data['valid']),len(data['test']),len(data['train'])+len(data['valid'])+len(data['test']),dataset_size))

### Now we generate noisy pngs. The image processing code is a tiny bit different than before

In [ ]:
import matplotlib.image
def save_images(settype):
    
    if settype.lower() != 'train' and settype.lower() != 'test' and settype.lower() !='valid':
        raise ValueError("settype must be 'train','valid',or 'test'")
    
    path = conf['project_dir'] + '/datasets/%s%s/images'%(settype.lower(),conf['suffix'])
    print(f"Saving images to path: {path}")
    
    #Create our output directory if it doesn't already exist
    if not os.path.exists(path):
        os.makedirs(path)
    
    for i in tqdm(range(0,len(data[settype.lower()]))):
        
        tmp = data[settype.lower()].iloc[i]
        
        '''Setting bins to (512,288) downsamples the image with 4x4 binning'''
        im = np.histogram2d(tmp['x'],tmp['y'],weights=tmp['vignetted_q'],bins=(512,288),range=((0,2048),(0,1152)))[0].T
        
        '''Add noise'''
        dark_idx = np.random.randint(0,200)
        im += darkDownSample[dark_idx]
        
        '''The colorscale (vmin and vmax) as well as how we define im depend on if we use linear or logarithmic
        colorscale images'''
        if conf['log_scale'] == False:
            matplotlib.image.imsave('%s/%s.png'%(path,tmp['index']), im, vmin=-100, vmax=600,cmap = 'jet')
        else:
            im[im<0] = 0 #We probably shouldn't do this but this allows for clean log scale images
            im = np.log10(im+1)
            matplotlib.image.imsave('%s/%s.png'%(path,tmp['index']), im, vmin=1.4, vmax=2.5,cmap = 'jet')

In [ ]:
for key in ['train','valid','test']:
    save_images(key)

In [ ]:
'''Ditto here: whether or not we use a log scale is determined from master_configuration.yaml'''
def save_labels(settype):
    if settype.lower() != 'train' and settype.lower() != 'test' and settype.lower() !='valid':
        raise ValueError("settype must be 'train','valid',or 'test'")

    path = conf['project_dir'] + '/datasets/%s%s/labels/'%(settype.lower(),conf['suffix'])
    print(f"Saving labels to path: {path}")
    
    if not os.path.exists(path):
        os.makedirs(path)

    for i in range(0,len(data[settype.lower()])):
        tmp = data[settype.lower()].iloc[i]
        series = tmp[['class_index','xBB', 'yBB', 'width','height'] + list(np.array([['p%sx'%(i), 'p%sy'%(i)] for i in range(0,conf['maxNumKeyPoints'])]).flatten())]
        with open(path+'%s.txt'%(tmp['index']), 'w') as f:
            series_str = ' '.join(map(str, series.values))
            f.write(series_str + '\n')
            f.close()

In [ ]:
for key in ['train','valid']:
    save_labels(key)

In [ ]:
# Save Noisy data files. The track shifts are randomized so we save separate sets for linear and log colorscale
for key in ['train','valid','test']:
    data[key].index = [i for i in range(0,len(data[key]))]
    outfile = path = conf['project_dir'] + "/data/%s%s.feather"%(key,conf['suffix'])
    print(f"Saving metadata to {outfile}")  
    data[key].to_feather(outfile)

### Now we can finally train on our realistic noisy sim and then compare the keypoints to truth in the part2 notebookcript running overnight, or sending your data to Jeff so he can train it on a GPU)

In [ ]:
yt.create_keypoint_config('../master_configuration.yaml')

In [ ]:
from ultralytics import YOLO

# Load a base model
model = YOLO('yolov8m-pose.yaml')  # load empty model. Can choose from yolov8{n,s,m,l,x}-pose.yaml. Letters are ordered from smallest model to largest

#Function to train YOLO
#The project field sets the directory where YOLO's trained weights will be assigned
model.train(data='/home/jeef/workspace/migKeyPoint/migKeyPoint/tutorial/configs/keypoint.yaml',project=conf['project'],epochs=1000,patience=25,imgsz=512,rect=True)
